**Import the necessary libraries.**

In [ ]:
import ccxt
import pandas as pd
import numpy as np
import datetime
import os

from sklearn.preprocessing import MinMaxScaler

**Configuration**

In [ ]:
SEQUENCE_LENGTH = 60  # Use the last 60 data points to predict the next one
SPLIT_RATIO = 0.8  # Use 80% of the data for training and 20% for testing
EPOCHS = 50
BATCH_SIZE = 64

**Data Collection**

In [ ]:
def fetch_ohlcv(symbol, timeframe='1h'):
    # Initialize the exchange
    exchange = ccxt.binance()
    
    # Calculate the date from 2 years ago
    two_years_ago = (datetime.datetime.utcnow() - datetime.timedelta(days=2*365)).isoformat()
    
    # Generate a Windows-compatible filename
    file_name = f"raw-{symbol.replace('/', '-').lower()}-data.csv"
    
    # Check if the dataset already exists
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
        since = exchange.parse8601(df.iloc[-1]['timestamp'])  # Get the last timestamp
    else:
        since = exchange.parse8601(two_years_ago)  # Start from 2 years ago
        df = pd.DataFrame()
    
    # Fetch OHLCV data
    new_data = pd.DataFrame()
    while True:
        try:
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since)
            if len(ohlcv) == 0:
                break
            since = ohlcv[-1][0] + 1  # Update the `since` variable to avoid fetching the same data
            # Correct the column names
            new_data = pd.concat([new_data, pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])], ignore_index=True)
        except Exception as e:
            print(f"Error fetching data: {e}")
            break
    
    # Remove overlapping data points from both dataframes
    if 'timestamp' in df.columns and 'timestamp' in new_data.columns:
        overlap_timestamp = new_data['timestamp'].min()
        df = df[df['timestamp'] < overlap_timestamp]
        new_data = new_data[new_data['timestamp'] >= overlap_timestamp]
    
    # Reset indices and append the new data
    df = df.reset_index(drop=True)
    new_data = new_data.reset_index(drop=True)
    df = pd.concat([df, new_data], ignore_index=True)
    
    df.to_csv(file_name, index=False)
    return df

**Fetching the Data for 'BTC/USDT'**

In [ ]:
# Fetching 2 years of 1-hour data for 'BTC/USDT' using the fixed function
data_fixed = fetch_ohlcv('BTC/USDT')
data_fixed.head()  # Display the first few rows of the data

**Data Preprocessing**

Normalize the Data

In [ ]:
# Extract the 'close' prices for normalization
close_prices = data_fixed['close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
normalized_data = scaler.fit_transform(close_prices)

Create Sequences of Data for LSTM

In [ ]:
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 60
X, y = create_sequences(normalized_data, SEQ_LENGTH)

Split the Data into Training and Testing Sets

In [ ]:
train_size = int(len(X) * 0.8)

X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

**LSTM Model Creation**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()

# First LSTM layer
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

# Second LSTM layer
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))

# Third LSTM layer
model.add(LSTM(50))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

Train the LSTM Model

In [ ]:
history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test))